In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import boxcox
from statsmodels.tsa.seasonal import STL

In [ ]:
src_url = 'https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv'
df = pd.read_csv(src_url, index_col='Date')
df.index = pd.to_datetime(df.index)
df = df[df.index >= datetime.strptime('2010-01-01', '%Y-%m-%d')]
df = df.resample('W').sum()

In [ ]:
df

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(18,14), sharey=True)

axs[0].plot(df['Wind'])
axs[0].plot(df['Wind'].rolling(4).mean().rolling(2).mean())

axs[1].plot(df['Wind'] - df['Wind'].rolling(8).mean())

In [ ]:
stl = STL(df['Wind'], period=7, seasonal=365)

In [ ]:
res = stl.fit()

In [ ]:
fig = res.plot()

In [ ]:
plt.plot(df['Wind'] - res.seasonal)

In [ ]:

plt.plot(df['Wind'].rolling(4).mean())